# 4.3 事後予測チェック

## インポート・設定

In [1]:
import dataclasses
import warnings
from numbers import Number

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Markdown
from scipy import signal, special, stats

sns.set_theme()
warnings.simplefilter("ignore")
SEED = 0

## a